<center><h1>Course 3: Robot Dynamics</h1></center>

**NOTE:**

The variable names were assigned according to the code library documentation.

The python code (Modern robotics library)is commented and mostly self-explanatory in conjunction with the book. An example use is provided with each function.

Information on installing and using the library is available at the code website, https://github.com/NxRLab/ModernRobotics.

<h2><b>Chapter 8 through 8.3, Dynamics of Open Chains</b></h2>

In [1]:
# These libraries are used in every exercise
import modern_robotics as mr
import numpy as np

<b>5.</b> An inexact model of the UR5 mass and kinematic properties is given below:

$
M_{01} =\begin{bmatrix}
1 &0  &0  &0 \\ 
0 &1  &0  &0 \\ 
0 &0  &1  &0.089159 \\ 
0 &0  &0  &1 
\end{bmatrix},
M_{12} =\begin{bmatrix}
0 &0  &1  &0.28 \\ 
0 &1  &0  &0.13585 \\ 
-1 &0  &0  &0 \\ 
0 &0  &0  &1 
\end{bmatrix},\\
M_{23} =\begin{bmatrix}
1 &0  &0  &0 \\ 
0 &1  &0  &-0.1197 \\ 
0 &0  &1  &0.395 \\ 
0 &0  &0  &1 
\end{bmatrix},
M_{34} =\begin{bmatrix}
0 &0  &1  &0 \\ 
0 &1  &0  &0 \\ 
-1 &0  &0  &0.14225 \\ 
0 &0  &0  &1 
\end{bmatrix},\\
M_{45} =\begin{bmatrix}
1 &0  &0  &0 \\ 
0 &1  &0  &0.093 \\ 
0 &0  &1  &0 \\ 
0 &0  &0  &1 
\end{bmatrix},
M_{56} =\begin{bmatrix}
1 &0  &0  &0 \\ 
0 &1  &0  &0 \\ 
0 &0  &1  &0.09465 \\ 
0 &0  &0  &1 
\end{bmatrix},\\
M_{67} =\begin{bmatrix}
1 &0  &0  &0 \\ 
0 &0  &1  &0.0823 \\ 
0 &-1  &0  &0 \\ 
0 &0  &0  &1 
\end{bmatrix}
$

$
G1 = diag([0.010267495893, 0.010267495893,  0.00666, 3.7, 3.7, 3.7])\\
\\
G2 = diag([0.22689067591, 0.22689067591, 0.0151074, 8.393, 8.393, 8.393])\\
\\
G3 = diag([0.049443313556, 0.049443313556, 0.004095, 2.275, 2.275, 2.275])\\
\\
G4 = diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])\\
\\
G5 = diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])\\
\\
G6 = diag([0.0171364731454, 0.0171364731454, 0.033822, 0.1879, 0.1879, 0.1879])
$

$
Slist = \begin{bmatrix}
0 &0  &0  &0  &0  &0 \\ 
0 &1  &1  &1  &0  &1 \\ 
1 &0  &0  &0  &-1  &0 \\ 
0 &-0.089159  &-0.089159  &-0.089159  &-0.10915  &0.005491 \\ 
0 &0  &0  &0  &0.81725  &0 \\ 
0 &0  &0.425  &0.81725  &0  &0.81725 
\end{bmatrix}\\
$

$
\theta = \begin{matrix}
0\\ \pi/6\\ \pi/4\\ \pi/3\\ \pi/2\\ 2\pi/3
\end{matrix},~~
\dot{\theta} = \begin{matrix}
0.2\\ 0.2\\ 0.2\\ 0.2\\ 0.2\\ 0.2
\end{matrix},~~
\ddot{\theta} = \begin{matrix}
0.1\\ 0.1\\ 0.1\\ 0.1\\ 0.1\\ 0.1
\end{matrix},~~
g = \begin{matrix}
0\\ 0\\ -9.81
\end{matrix},~~
F_{tip} = \begin{matrix}
0.1\\ 0.1\\ 0.1\\ 0.1\\ 0.1\\ 0.1
\end{matrix}
$

use the function `InverseDynamics`  in the given software to calculate the required joint forces/torques of  the robot. The maximum allowable error for any number is $0.01$, so give enough decimal places where necessary.


<img src="images/UR5_model_6axis_robot.PNG" width=300 height=300 />

* <span style="color:gray"><b>thetalist:</b></span> n-vector of join variables $\theta$.
* <span style="color:gray"><b>dthetalist:</b></span> n-vector of join velocities $\dot{\theta}$.
* <span style="color:gray"><b>ddthetalist:</b></span> n-vector of join accelerations $\ddot{\theta}$.
* <span style="color:gray"><b>g:</b></span> Gravity vector $g$.
* <span style="color:gray"><b>Ftip:</b></span> Wrench $F_{tip}$ applied by the end-effector expressed in frame $\{n+1\}$. 
* <span style="color:gray"><b>Mlist:</b></span> List of link frames $\{i\}$ relative to $\{i-1\}$ at the home position.
* <span style="color:gray"><b>Glist:</b></span> Spatial inertia matrices $G_i$ of the links.
* <span style="color:gray"><b>Slist:</b></span> Screw axes $S_i$ of the joints in a space frame.
* <span style="color:gray"><b>taulist:</b></span> The n-vector $\tau$ of required joint forces/torques

In [34]:
import math

# Data can be found here:
# https://d3c33hcgiwev3.cloudfront.net/_be39b6c4a1f5e8e9109587e3e5a1ae56_UR5_parameter.py?Expires=1653696000&Signature=UPni7nFKCHq8F6vdeFPCY4qBacH8lH3VSjQdkzdG-MsPd1os-EzESeJatjROTXruH9vsNUwoX8oqr6cHo5O~WaZZr4I6H5JKwh-PBG~NF9OPixyPHkSqvMMrjvljHCMhJtTp1UgUiPnh07rLo3yqNVi9OVQPNSWcGcmOGpi1w6I_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A

M01 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.089159], [0, 0, 0, 1]]
M12 = [[0, 0, 1, 0.28], [0, 1, 0, 0.13585], [-1, 0, 0, 0], [0, 0, 0, 1]]
M23 = [[1, 0, 0, 0], [0, 1, 0, -0.1197], [0, 0, 1, 0.395], [0, 0, 0, 1]]
M34 = [[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0.14225], [0, 0, 0, 1]]
M45 = [[1, 0, 0, 0], [0, 1, 0, 0.093], [0, 0, 1, 0], [0, 0, 0, 1]]
M56 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.09465], [0, 0, 0, 1]]
M67 = [[1, 0, 0, 0], [0, 0, 1, 0.0823], [0, -1, 0, 0], [0, 0, 0, 1]]
G1 = np.diag([0.010267495893, 0.010267495893,  0.00666, 3.7, 3.7, 3.7])
G2 = np.diag([0.22689067591, 0.22689067591, 0.0151074, 8.393, 8.393, 8.393])
G3 = np.diag([0.049443313556, 0.049443313556, 0.004095, 2.275, 2.275, 2.275])
G4 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G5 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G6 = np.diag([0.0171364731454, 0.0171364731454, 0.033822, 0.1879, 0.1879, 0.1879])
Glist = [G1, G2, G3, G4, G5, G6]
Mlist = [M01, M12, M23, M34, M45, M56, M67] 
Slist = [[0,         0,         0,         0,        0,        0],
         [0,         1,         1,         1,        0,        1],
         [1,         0,         0,         0,       -1,        0],
         [0, -0.089159, -0.089159, -0.089159, -0.10915, 0.005491],
         [0,         0,         0,         0,  0.81725,        0],
         [0,         0,     0.425,   0.81725,        0,  0.81725]]

thetalist=[0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*(np.pi/3)]

dthetalist=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

ddthetalist=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

g=[0, 0, -9.81] 

Ftip=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

taulist = mr.InverseDynamics(thetalist,dthetalist,ddthetalist,
g,Ftip,Mlist,Glist,Slist)

#avoid scientific notation
for item in taulist:
    print("{:.4f}".format(item),end=", ")


0.0128, -41.1477, -3.7809, 0.0323, 0.0370, 0.1034, 

<h2><b>Chapter 8.5-8.7 and 8.9, Dynamics of Open Chains</b></h2>

<b>1 - 5</b>.A robot system (UR5) is defined as: 

(**Use parameters in previous exercise in this notebook**)

Use the function:
* `MassMatrix` to calculate the numerical inertia matrix of the robot. 

* `VelQuadraticForces` to calculate the Coriolis and centripetal terms in the robot's dynamics (This function calls InverseDynamics with $g = 0, F_{tip} = 0, and \ddot{\theta} = 0$)

* `GravityForces` to calculate the joint forces/torques required to overcome gravity

* `EndEffectorForces` to calculate the joint forces/torques required to generate the wrench $F_{tip}$. (This function calls InverseDynamics with $\dot{\theta} = \ddot{\theta} = 0 and F_{tip} = 0$)

* `ForwardDynamics` to find the joint acceleration (This function computes $\ddot{\theta}$ by solving: $M(\theta)\ddot{\theta}=\tau - c(\theta,\dot{\theta}) - g(\theta) - J^T(\theta)F_{tip}$ )

The maximum allowable error for any number is $0.01$, so give enough decimal places where necessary.

* <span style="color:gray"><b>thetalist:</b></span> n-vector of join variables $\theta$.
* <span style="color:gray"><b>Mlist:</b></span> List of link frames $\{i\}$ relative to $\{i-1\}$ at the home position.
* <span style="color:gray"><b>Glist:</b></span> Spatial inertia matrices $G_i$ of the links.
* <span style="color:gray"><b>Slist:</b></span> Screw axes $S_i$ of the joints in a space frame.
* <span style="color:gray"><b>M:</b></span> The numerical inertia matrix $M(\theta)$ of an n-joint serial chain at the given configuration $\theta$.
* <span style="color:gray"><b>c:</b></span> The vector $c(\theta, \dot{\theta})$ of Coriolis and centripetal terms for a given $\theta$ and $\dot{\theta}$
* <span style="color:gray"><b>grav:</b></span> The joint forces/torques required to balance gravity at $\theta$
* <span style="color:gray"><b>JTFtip:</b></span> The joint forces and torques $J^T(θ)F_{tip}$ required to create the end-effector force Ftip
* <span style="color:gray"><b>ddthetalist:</b></span> The resulting joint acceleration $\ddot{\theta}$


In [48]:
import math

M01 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.089159], [0, 0, 0, 1]]
M12 = [[0, 0, 1, 0.28], [0, 1, 0, 0.13585], [-1, 0, 0, 0], [0, 0, 0, 1]]
M23 = [[1, 0, 0, 0], [0, 1, 0, -0.1197], [0, 0, 1, 0.395], [0, 0, 0, 1]]
M34 = [[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0.14225], [0, 0, 0, 1]]
M45 = [[1, 0, 0, 0], [0, 1, 0, 0.093], [0, 0, 1, 0], [0, 0, 0, 1]]
M56 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.09465], [0, 0, 0, 1]]
M67 = [[1, 0, 0, 0], [0, 0, 1, 0.0823], [0, -1, 0, 0], [0, 0, 0, 1]]
G1 = np.diag([0.010267495893, 0.010267495893,  0.00666, 3.7, 3.7, 3.7])
G2 = np.diag([0.22689067591, 0.22689067591, 0.0151074, 8.393, 8.393, 8.393])
G3 = np.diag([0.049443313556, 0.049443313556, 0.004095, 2.275, 2.275, 2.275])
G4 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G5 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G6 = np.diag([0.0171364731454, 0.0171364731454, 0.033822, 0.1879, 0.1879, 0.1879])
Glist = [G1, G2, G3, G4, G5, G6]
Mlist = [M01, M12, M23, M34, M45, M56, M67] 
Slist = [[0,         0,         0,         0,        0,        0],
         [0,         1,         1,         1,        0,        1],
         [1,         0,         0,         0,       -1,        0],
         [0, -0.089159, -0.089159, -0.089159, -0.10915, 0.005491],
         [0,         0,         0,         0,  0.81725,        0],
         [0,         0,     0.425,   0.81725,        0,  0.81725]]

thetalist=[0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*(np.pi/3)]

dthetalist=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

ddthetalist=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

g=[0, 0, -9.81] 

Ftip=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

taulist=[0.0128, -41.1477, -3.7809, 0.0323, 0.0370, 0.1034]


M = mr.MassMatrix(thetalist, Mlist, Glist, Slist)
c = mr.VelQuadraticForces(thetalist,dthetalist,Mlist,Glist,Slist)
grav = mr.GravityForces(thetalist,g,Mlist,Glist,Slist)
JTFtip = mr.EndEffectorForces(thetalist,Ftip,Mlist,Glist,Slist)
ddthetalist = mr.ForwardDynamics(thetalist,dthetalist,taulist,g,Ftip,Mlist,Glist,Slist)


#avoid scientific notation
np.set_printoptions(suppress=True)
print("1. Numerical inertia matrix of the robot:",np.round_(M,5),sep='\n\n',end='\n\n\n')

print("2. Coriolis and centripetal terms in the robot's dynamics:",np.round_(c,5),sep='\n\n',end='\n\n\n')

print("3.  joint forces/torques required to overcome gravity:",np.round_(grav,5),sep='\n\n',end='\n\n\n')

print("4.  joint forces/torques required to generate the wrench Ftip:",np.round_(JTFtip,5),sep='\n\n',end='\n\n\n')

print("5.  joint acceleration:",np.round_(ddthetalist,5),sep='\n\n',end='\n\n')

1. Numerical inertia matrix of the robot:

[[ 2.19777  0.27228  0.06803 -0.00648  0.17022 -0.01212]
 [ 0.27228  3.55369  1.31041  0.24034 -0.00723  0.     ]
 [ 0.06803  1.31041  0.83725  0.24764 -0.00723  0.     ]
 [-0.00648  0.24034  0.24764  0.25368 -0.00723  0.     ]
 [ 0.17022 -0.00723 -0.00723 -0.00723  0.24073  0.     ]
 [-0.01212  0.       0.       0.       0.       0.01714]]


2. Coriolis and centripetal terms in the robot's dynamics:

[-0.11745 -0.01071  0.03165 -0.01477  0.02339  0.00287]


3.  joint forces/torques required to overcome gravity:

[  0.      -41.59673  -3.93591   0.12337   0.        0.     ]


4.  joint forces/torques required to generate the wrench Ftip:

[-0.13875 -0.07721 -0.12229 -0.14908 -0.02536  0.1    ]


5.  joint acceleration:

[0.10001 0.09995 0.10017 0.09985 0.10019 0.10191]



<h2><b>Chapter 9 through 9.3, Trajectory Generation</b></h2>

<b>5</b>. Given a total travel time $T=5$ and the current time $t=3$, use the function `QuinticTimeScaling` in the given software to calculate the current path parameter $s$, with at least 2 decimal places, corresponding to a motion that begins and ends at zero velocity and acceleration.

* <span style="color:gray"><b>Tf: </b></span>Total time of the motion $T_f$ in seconds from rest to rest

* <span style="color:gray"><b>t: </b></span> The current time $t$ satisfying $0 ≤ t ≤ T_f$

* <span style="color:gray"><b>s: </b></span> The path parameter $s(t)$ corresponding to a fifth-order polynomial motion that begins (at $s(0) = 0$) and ends (at $s(T_f) = 1$) at zero velocity and zero acceleration.

In [50]:
import math

Tf = 5
t = 3

s = mr.QuinticTimeScaling(Tf, t)

print(round(s,4))

0.6826


<b>6</b>. Use the function `ScrewTrajectory` in the given software to calculate a trajectory as a list of $N=10 SE(3)$ matrices, where each matrix represents the configuration of the end-effector at an instant in time. The first matrix is:

$$
X_{start}=\begin{bmatrix}
 1&0  &0  &0 \\ 
 0&1  &0  &0 \\ 
 0&0  &1  &0 \\ 
 0&0  &0  &1 
\end{bmatrix}
$$

and the 10th matrix is:

$$
X_{end}=\begin{bmatrix}
 0&0  &1  &1 \\ 
 1&0  &0  &2 \\ 
 0&1  &0  &3 \\ 
 0&0  &0  &1 
\end{bmatrix}
$$

The motion is along a constant screw axis and the duration is $T_f=10$. The parameter $method$ equals 3 for a cubic time scaling. Give the 9th matrix (one before $X_{end}$) in the returned trajectory.  The maximum allowable error for any matrix entry is $0.01$, so give enough decimal places where necessary.


* <span style="color:gray"><b>Xstart: </b></span> The initial end-effector configuration $X_{start} ~\epsilon ~SE(3)$
* <span style="color:gray"><b>Xend: </b></span> The final end-effector configuration $X_{end}$.
* <span style="color:gray"><b>Tf: </b></span> Total time of the motion $T_f$ in seconds from rest to rest. 
* <span style="color:gray"><b>N: </b></span> The number of points $N ≥ 2$ in the discrete representation of the trajectory.
* <span style="color:gray"><b>method: </b></span> The time-scaling method, where 3 indicates cubic (third-order polynomial) time scaling and 5 indicates quintic (fifth-order polynomial) time scaling.
* <span style="color:gray"><b>traj: </b></span> The discretized trajectory as a list of $N$ matrices in $SE(3)$ separated in time by $T_f/(N −
1)$. The first in the list is $X_{start}$ and the $N_{th}$ is $X_{end}$.

In [54]:
import math

Xstart = np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

Xend = np.array([[0, 0, 1, 1],
                 [1, 0, 0, 2],
                 [0, 1, 0, 3],
                 [0, 0, 0, 1]])
Tf = 10
N = 10
method = 3 #cubic time scaling

traj = mr.ScrewTrajectory(Xstart,Xend,Tf,N,method)

print ("9th trajectory: ",np.round_(traj[8],4),sep="\n",end="\n\n")


9th trajectory: 
[[ 0.0423 -0.0406  0.9983  0.9331]
 [ 0.9983  0.0423 -0.0406  1.972 ]
 [-0.0406  0.9983  0.0423  2.8891]
 [ 0.      0.      0.      1.    ]]

